In [2]:
import re
from open_spiel.python.algorithms import cfr, outcome_sampling_mccfr, expected_game_score, exploitability, get_all_states_with_policy
import pyspiel
from open_spiel.python.algorithms.cfr import CFRSolver
import pandas as pd
import pickle
import cProfile, pstats


In [244]:
# GAME_PATH = '/apps/open_spiel/sats/python/odd_licenses.json'
# GAME_PATH = 'very_small_sats_complete_symmetric.json'
# GAME_PATH = '/apps/open_spiel/sats/python/bluff.json'
GAME_PATH = '/apps/open_spiel/sats/python/mixed.json'

In [3]:
GAME_NAME = 'mixed'
PYSATS = '/apps/sats/python'
!python {PYSATS}/pysats.py --config_file {PYSATS}/{GAME_NAME}.yml --seed 777 --output_file {PYSATS}/{GAME_NAME}.json
# GAME_NAME = f'{PYSATS}/{GAME_NAME}'
!mv {GAME_PATH} /apps/open_spiel/configs/{GAME_NAME}.json

In [6]:
!python /apps/cluster_open_spiel/open_spiel/python/examples/ubc_mccfr_cpp_example.py --filename {GAME_NAME}.json --iterations 5000 --output {GAME_NAME}_mccfr --game python_clock_auction --python true --solver mccfr 

I0419 20:35:13.002578 140289505829376 clock_auction_parser.py:14] Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
I0419 20:35:13.002713 140289505829376 clock_auction_parser.py:18] CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
I0419 20:35:13.002764 140289505829376 clock_auction_parser.py:21] Parsing configuration from /apps/open_spiel/configs/mixed.json
I0419 20:35:13.002989 140289505829376 clock_auction_parser.py:100] Done config parsing
I0419 20:35:13.003064 140289505829376 clock_auction.py:133] Folding randomnesss...
I0419 20:35:13.003258 140289505829376 ubc_mccfr_cpp_example.py:167] Using python implementations
I0419 20:35:13.003382 140289505829376 ubc_mccfr_cpp_example.py:178] Using MCCFR solver
I0419 20:35:13.003446 140289505829376 ubc_mccfr_cpp_example.py:183] Using external sampling
I0419 20:35:13.003664 140289505829376 ubc_mccfr_cpp_example.py:223] Starting iteration 0
I0419 20:35:28.372768 140289505829376 ubc_mccfr_cpp_example.p

In [33]:
# def prof():
#     game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))
#     num_iterations = 500
#     cfr_solver = CFRSolver(game)
#     for i in range(num_iterations):
#         cfr_solver.evaluate_and_update_policy()


# profiler = cProfile.Profile()
# profiler.enable()
# prof()
# profiler.disable()

New state not in cache, processed... 2 outcomes into 2 outcomes
New state not in cache, processed... 1 outcomes into 1 outcomes
New state not in cache, processed... 6 outcomes into 2 outcomes
New state not in cache, processed... 1 outcomes into 1 outcomes
New state not in cache, processed... 1 outcomes into 1 outcomes
New state not in cache, processed... 2 outcomes into 1 outcomes
New state not in cache, processed... 6 outcomes into 2 outcomes
New state not in cache, processed... 2 outcomes into 1 outcomes
New state not in cache, processed... 24 outcomes into 2 outcomes
New state not in cache, processed... 6 outcomes into 2 outcomes
New state not in cache, processed... 2 outcomes into 1 outcomes
New state not in cache, processed... 2 outcomes into 2 outcomes
New state not in cache, processed... 1 outcomes into 1 outcomes
New state not in cache, processed... 2 outcomes into 1 outcomes
New state not in cache, processed... 1 outcomes into 1 outcomes
New state not in cache, processed... 1 

IOStream.flush timed out


In [14]:
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()


         286545250 function calls (273334744 primitive calls) in 459.696 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  459.696  229.848 /home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3215(run_code)
        2    0.000    0.000  459.696  229.848 {built-in method builtins.exec}
        1    0.000    0.000  459.696  459.696 /tmp/ipykernel_3690/2409957485.py:11(<module>)
        1    0.005    0.005  459.695  459.695 /tmp/ipykernel_3690/2409957485.py:1(prof)
     5000    0.163    0.000  450.008    0.090 /apps/open_spiel/open_spiel/python/algorithms/cfr.py:419(evaluate_and_update_policy)
11487165/10000  196.588    0.000  422.574    0.042 /apps/open_spiel/open_spiel/python/algorithms/cfr.py:240(_compute_counterfactual_regret_for_player)
 11488667   62.552    0.000  111.101    0.000 /apps/open_spiel/open_spiel/python/games/clock_auction.py:21

In [ ]:
game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))


def state_to_final_python(game, s):
    '''Convert a state into unique final outcomes (but not caring about bidding being different in the middle). i.e., the allocation and the types and the price are all the same'''
    info_state_string = f'Price: {s.posted_prices[-1]}\n'
    info_state_string += f'Final bids: {s.get_allocation()}\n'
    info_state_string += f'Round: {s.round}\n'
    return info_state_string


all_states = get_all_states_with_policy.get_all_info_states_with_policy(
    game,
    depth_limit=-1,
    include_terminals=True,
    policy=None,
    to_string=lambda s: state_to_final_python(game, s)
)


In [ ]:
game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))
solver = pickle.load(open('april6_t1/cfr_9.pkl', 'rb'))
policy = solver.average_policy()

all_states = get_all_states_with_policy.get_all_info_states_with_policy(
    game,
    depth_limit=-1,
    include_terminals=True,
    policy=policy,
    to_string=lambda s: state_to_final_python(game, s)
)


In [ ]:
game.new_initial_state().get_game()

In [ ]:
import pprofile
profiler = pprofile.Profile()
with profiler:
    prof()
profiler.dump_stats('output_april17.prof')

In [ ]:
game = pyspiel.load_game("python_clock_auction", dict(filename='april6_t1.json'))

In [ ]:
game.make_py_observer().string_from.cache_info()